# Global Explainability Evaluation

Structure:
- Computing explanations for all models and datasets
- evaluating global faithfulness (comprehensiveness)
- evaluating global plausibility (IOU scores)

## Cloning TransformerExplainability Repo

In [ ]:
!git clone https://github.com/henningheyen/ICLR-Submission.git

In [4]:
%cd ICLR-Submission

/content/ICLR-Submission


In [ ]:
# Installing Dependencies
!pip install lime
!pip install transformers
!pip install sentencepiece
!pip install datasets

## Setup

In [6]:
from utils import make_test_set_esnli, make_test_set_mnli, make_test_set_cose
from model import ModelNLI, ModelZSC
from explainer import Explainer
import numpy as np


In [8]:
# number of explanations
NUM_EXPL = 100

#importing dataset
dataset_esnli = make_test_set_esnli(size=NUM_EXPL, remove_bad_explanations=True) # 9830 is dev size esnli (originally 9842 but 12 instance are wrongly annotated)
dataset_mnli = make_test_set_mnli(size=NUM_EXPL) # 9815 is dev size mnli
dataset_cose = make_test_set_cose(size=NUM_EXPL, remove_bad_explanations=True) # 718 (originally 1221 is dev size cose but 503 instances have bad explanations, i.e. the whole question highlighted))

In [9]:
#calculating average explanation length per dataset

full_dataset_esnli = make_test_set_esnli(size=9830, remove_bad_explanations=True) # 9830 is dev size esnli (originally 9842 but 12 instance are wrongly annotated)
full_dataset_cose = make_test_set_cose(size=718, remove_bad_explanations=True) # 718 (originally 1221 is dev size cose but 503 instances have bad explanations, i.e. the whole question highlighted))

len_esnli = [len(full_dataset_esnli['extractive_explanation'][i])/(len(full_dataset_esnli['sentence_pairs'][i][0].split())+len(full_dataset_esnli['sentence_pairs'][i][1].split())) for i in range(9830)]
len_cose = [len(full_dataset_cose['extractive_explanation'][i])/len(full_dataset_cose['question'][i].split()) for i in range(718)]

avg_len_esnli = np.mean(len_esnli)
avg_len_cose = np.mean(len_cose)

print('average explanation input ratio e-SNLI: ', avg_len_esnli)
print('average explanation input ratio CoS-e: ', avg_len_cose)

average explanation input ratio e-SNLI:  0.19858092123245383
average explanation input ratio CoS-e:  0.2605110957380536


## Calculating Explanations

### Natural Language Inference

In [16]:
# Natural Language Inference
xsmall_nli = ModelNLI(model_name='cross-encoder/nli-deberta-v3-xsmall')
small_nli = ModelNLI(model_name='cross-encoder/nli-deberta-v3-small')
base_nli = ModelNLI(model_name='cross-encoder/nli-deberta-v3-base')
large_nli = ModelNLI(model_name='cross-encoder/nli-deberta-v3-large')

models_nli = [
    xsmall_nli,
    small_nli,
    base_nli,
    large_nli
]

model_names_nli = [
    'xsmall_nli',
    'small_nli',
    'base_nli',
    'large_nli'
]


#### MNLI

In [11]:
explainer_mnli = Explainer(class_names=['contradiction', 'entailment', 'neutral'])

In [ ]:
%%time

# Computing LIME explanations on xsmall
explanations_xsmall_mnli = explainer_mnli.compute_explanations(
    sentences = dataset_mnli['sentence_pairs'],
    model=xsmall_nli,
    num_samples=100,
    task='NLI',
    )

CPU times: user 6min 28s, sys: 1min 38s, total: 8min 7s
Wall time: 2min 3s


2min 3s

In [ ]:
%%time

# Computing LIME explanations on small
explanations_small_mnli = explainer_mnli.compute_explanations(
    sentences = dataset_mnli['sentence_pairs'],
    model=small_nli,
    num_samples=100,
    task='NLI',
    )

CPU times: user 8min 40s, sys: 1min 57s, total: 10min 37s
Wall time: 2min 40s


2min 40s

In [ ]:
%%time

# Computing LIME explanations on base
explanations_base_mnli = explainer_mnli.compute_explanations(
    sentences = dataset_mnli['sentence_pairs'],
    model=base_nli,
    num_samples=100,
    task='NLI',
    )

CPU times: user 17min 20s, sys: 3min 54s, total: 21min 15s
Wall time: 5min 20s


5min 20s


In [ ]:
%%time

# Computing LIME explanations on large
explanations_large_mnli = explainer_mnli.compute_explanations(
    sentences = dataset_mnli['sentence_pairs'],
    model=large_nli,
    num_samples=100,
    task='NLI',
    )

CPU times: user 54min 8s, sys: 8min 12s, total: 1h 2min 20s
Wall time: 15min 38s


15min 38s

In [13]:
explanations_mnli = [
    explanations_xsmall_mnli,
    explanations_small_mnli,
    explanations_base_mnli,
    explanations_large_mnli,
    ]

#### e-SNLI

In [ ]:
explainer_esnli = Explainer(class_names=['contradiction', 'entailment', 'neutral'])

In [ ]:
%%time

# Computing LIME explanations on xsmall
explanations_xsmall_esnli = explainer_esnli.compute_explanations(
    sentences = dataset_esnli['sentence_pairs'],
    model=xsmall_nli,
    num_samples=100,
    task='NLI',
    )

CPU times: user 5min 38s, sys: 490 ms, total: 5min 39s
Wall time: 1min 25s


1min 8s

In [ ]:
%%time

# Computing LIME explanations on small
explanations_small_esnli = explainer_esnli.compute_explanations(
    sentences = dataset_esnli['sentence_pairs'],
    model=small_nli,
    num_samples=100,
    task='NLI',
    )

CPU times: user 8min 20s, sys: 1.5 s, total: 8min 21s
Wall time: 2min 7s


1min 40s

In [ ]:
%%time

# Computing LIME explanations on base
explanations_base_esnli = explainer_esnli.compute_explanations(
    sentences = dataset_esnli['sentence_pairs'],
    model=base_nli,
    num_samples=100,
    task='NLI',
    )

CPU times: user 17min 2s, sys: 6.13 s, total: 17min 8s
Wall time: 4min 20s


3min 35s

In [ ]:
%%time

# Computing LIME explanations on large
explanations_large_esnli = explainer_esnli.compute_explanations(
    sentences = dataset_esnli['sentence_pairs'],
    model=large_nli,
    num_samples=100,
    task='NLI',
    )

CPU times: user 56min 14s, sys: 8min 50s, total: 1h 5min 5s
Wall time: 16min 35s


12min 45s

In [ ]:
explanations_esnli = [
    explanations_xsmall_esnli,
    explanations_small_esnli,
    explanations_base_esnli,
    explanations_large_esnli,
    ]

### Zero Shot Classification

In [ ]:
# Zero Shot Classification
xsmall_zsc = ModelZSC(model_name='cross-encoder/nli-deberta-v3-xsmall')
small_zsc = ModelZSC(model_name='cross-encoder/nli-deberta-v3-small')
base_zsc = ModelZSC(model_name='cross-encoder/nli-deberta-v3-base')
large_zsc = ModelZSC(model_name='cross-encoder/nli-deberta-v3-large')

models_zsc = [
    xsmall_zsc,
    small_zsc,
    base_zsc,
    large_zsc,
]

model_names_zsc = [
    'xsmall_zsc',
    'small_zsc',
    'base_zsc',
    'large_zsc',
]

#### CoS-e

In [ ]:
# no class names becuase of zero shot classification setting
explainer_zsc = Explainer()

In [ ]:
%%time

# Computing LIME explanations on xsmall
explanations_xsmall_cose = explainer_zsc.compute_explanations(
   sentences = dataset_cose['question'],
   model=xsmall_zsc,
   num_samples=100,
   class_names_list=dataset_cose['candidate_labels_list']
   )

CPU times: user 2h 29min 35s, sys: 17.8 s, total: 2h 29min 53s
Wall time: 37min 27s


34min 35s

In [ ]:
%%time

# Computing LIME explanations on small
explanations_small_cose = explainer_zsc.compute_explanations(
   sentences = dataset_cose['question'],
   model=small_zsc,
   num_samples=100,
   class_names_list=dataset_cose['candidate_labels_list']
   )

CPU times: user 3h 16min 5s, sys: 14.7 s, total: 3h 16min 19s
Wall time: 49min 2s


44min 28s

In [ ]:
%%time

# Computing LIME explanations on base
explanations_base_cose = explainer_zsc.compute_explanations(
   sentences = dataset_cose['question'],
   model=base_zsc,
   num_samples=100,
   class_names_list=dataset_cose['candidate_labels_list']
   )

CPU times: user 6h 18min 23s, sys: 28.4 s, total: 6h 18min 51s
Wall time: 1h 34min 38s


1h 27min 7s

In [ ]:
%%time

# Computing LIME explanations on large
explanations_large_cose = explainer_zsc.compute_explanations(
   sentences = dataset_cose['question'],
   model=large_zsc,
   num_samples=100,
   class_names_list=dataset_cose['candidate_labels_list']
   )

CPU times: user 18h 34min 45s, sys: 4min 1s, total: 18h 38min 46s
Wall time: 4h 39min 25s


4h 35min 50s

In [ ]:
explanations_cose = [
    explanations_xsmall_cose,
    explanations_small_cose,
    explanations_base_cose,
    explanations_large_cose,
    ]

## Faithfulness (Comprehensiveness)

### Natural Language Inference

#### MNLI

In [17]:
# Calculating aggregated comprehensiveness on 100 explanations
comp_list_mnli = []

for i, model in enumerate(models_nli):

    print('model: ', model_names_nli[i])

    comp_agg = [explainer_mnli.aggregated_metric(metric='comprehensiveness', explanation=explanations_mnli[i][j], sentence=dataset_mnli['sentence_pairs'][j], predict=model.predict, verbose=False, bins=[0.1,0.3,0.5], task='NLI') for j in range(NUM_EXPL)]

    comp_list_mnli.append(comp_agg)


In [ ]:
#new
for i, model in enumerate(models_nli):
    print(f'MNLI average aggregated comprehensiveness {model_names_nli[i]}: ', np.mean(comp_list_mnli[i]))


- MNLI average aggregated comprehensiveness xsmall_nli:  0.78477615
- MNLI average aggregated comprehensiveness small_nli:  0.81709516
- MNLI average aggregated comprehensiveness base_nli:  0.79582304
- MNLI average aggregated comprehensiveness large_nli:  0.82296914


#### e-SNLI

In [ ]:
# Calculating aggregated comprehensiveness on 100 explanations
comp_list_esnli = []

for i, model in enumerate(models_nli):

    print('model: ', model_names_nli[i])

    comp_agg = [explainer_esnli.aggregated_metric(metric='comprehensiveness', explanation=explanations_esnli[i][j], sentence=dataset_esnli['sentence_pairs'][j], predict=model.predict, verbose=False, bins=[0.1,0.3,0.5], task='NLI') for j in range(NUM_EXPL)]

    comp_list_esnli.append(comp_agg)



In [ ]:
# before
for i, model in enumerate(models_nli):
    print(f'e-SNLI average aggregated comprehensiveness {model_names_nli[i]}: ', np.mean(comp_list_esnli[i]))


- e-SNLI average aggregated comprehensiveness xsmall_nli:  0.7259713
- e-SNLI average aggregated comprehensiveness small_nli:  0.72445035
- e-SNLI average aggregated comprehensiveness base_nli:  0.76369643
- e-SNLI average aggregated comprehensiveness large_nli:  0.77805173


### Zero Shot Classification

### CoS-e

In [ ]:
# Calculating aggregated comprehensiveness on 100 explanations
comp_list_cose = []

for i, model in enumerate(models_zsc):

    print('model: ', model_names_zsc[i])

    comp_agg = [explainer_zsc.aggregated_metric(metric='comprehensiveness', explanation=explanations_cose[i][j], sentence=dataset_cose['question'][j], predict=model.predict, verbose=False, bins=[0.1,0.3,0.5], task='ZSC', candidate_labels=dataset_cose['candidate_labels_list'][j]) for j in range(NUM_EXPL)]

    comp_list_cose.append(comp_agg)



In [ ]:
for i, model in enumerate(models_zsc):
    print(f'CoS-e average aggregated comprehensiveness {model_names_zsc[i]}: ', np.mean(comp_list_cose[i]))



- CoS-e average aggregated comprehensiveness xsmall_zsc:  0.30408690341748296
- CoS-e average aggregated comprehensiveness small_zsc:  0.31567202631694574
- CoS-e average aggregated comprehensiveness base_zsc:  0.35605754521364963
- CoS-e average aggregated comprehensiveness large_zsc:  0.3910369690948089



## Plausibility (IOU Scores)

### e-SNLI

In [ ]:
explanation_tokens_lists = [explainer_esnli.get_explanation_list(explanations_esnli[i], top_percent=avg_len_esnli) for i in range(len(models_nli))]
ground_truth_list = dataset_esnli['extractive_explanation']

iou_scores_esnli = []

for i, model in enumerate(models_nli):
  iou_scores_esnli.append([explainer_esnli.compute_instance_iou(explanation_tokens_lists[i][j], ground_truth_list[j]) for j in range(NUM_EXPL)])

for i, model in enumerate(models_nli):
    print(f'{model_names_nli[i]} macro_iou for {NUM_EXPL} number of explanations (CoS-e): ', np.mean(iou_scores_esnli[i]))


- xsmall_nli macro_iou for 100 number of explanations (e-SNLI):  0.27125388337153045
- small_nli macro_iou for 100 number of explanations (e-SNLI):  0.25177114388879096
- base_nli macro_iou for 100 number of explanations (e-SNLI):  0.24971816908581615
- large_nli macro_iou for 100 number of explanations (e-SNLI):  0.24985357779475428


### CoS-e

In [ ]:
explanation_tokens_lists = [explainer_zsc.get_explanation_list(explanations_cose[i], top_percent=avg_len_cose) for i in range(len(models_zsc))]
ground_truth_list = dataset_cose['extractive_explanation']

iou_scores_cose = []

for i, model in enumerate(models_zsc):
  iou_scores_cose.append([explainer_zsc.compute_instance_iou(explanation_tokens_lists[i][j], ground_truth_list[j]) for j in range(NUM_EXPL)])

for i, model in enumerate(models_zsc):
    print(f'{model_names_zsc[i]} macro_iou for {NUM_EXPL} number of explanations (CoS-e): ', np.mean(iou_scores_cose[i]))


- xsmall_zsc macro_iou for 100 number of explanations (CoS-e):  0.23265864492404734
- small_zsc macro_iou for 100 number of explanations (CoS-e):  0.23059835798876044
- base_zsc macro_iou for 100 number of explanations (CoS-e):  0.23547574449114694
- large_zsc macro_iou for 100 number of explanations (CoS-e):  0.23037491476531724


# Serialise objects

In [ ]:
explainability_results = {}

In [ ]:
explainability_results['mnli'] = {
        #'dataset': dataset_mnli,
        'xsmall': {
            'explanations_as_list':[explanation.as_list(label= explanation.top_labels[0]) for explanation in explanations_xsmall_mnli],
            'faithfulness': {
                'comprehensiveness': comp_list_mnli[0],
                'macro_comprehensiveness': np.mean(comp_list_mnli[0]),
            },
        },
        'small': {
            'explanations_as_list':[explanation.as_list(label= explanation.top_labels[0]) for explanation in explanations_small_mnli],
            'faithfulness': {
                'comprehensiveness': comp_list_mnli[1],
                'macro_comprehensiveness': np.mean(comp_list_mnli[1]),
            },
        },
        'base': {
            'explanations_as_list':[explanation.as_list(label= explanation.top_labels[0]) for explanation in explanations_base_mnli],
            'faithfulness': {
                'comprehensiveness': comp_list_mnli[2],
                'macro_comprehensiveness': np.mean(comp_list_mnli[2]),
            },
        },
        'large': {
            'explanations_as_list':[explanation.as_list(label= explanation.top_labels[0]) for explanation in explanations_large_mnli],
            'faithfulness': {
                'comprehensiveness': comp_list_mnli[3],
                'macro_comprehensiveness': np.mean(comp_list_mnli[3]),
            },
        },
  }

In [ ]:
explainability_results['esnli'] = {
        #'dataset': dataset_esnli,
        'xsmall': {
            'explanations_as_list':[explanation.as_list(label= explanation.top_labels[0]) for explanation in explanations_xsmall_esnli],
            'faithfulness': {
                'comprehensiveness': comp_list_esnli[0],
                'macro_comprehensiveness': float(np.mean(comp_list_esnli[0])),
            },
            'plausibility': {
                'iou': iou_scores_esnli[0],
                'macro_iou': float(np.mean(iou_scores_esnli[0])),
            },
        },
        'small': {
            'explanations_as_list':[explanation.as_list(label= explanation.top_labels[0]) for explanation in explanations_small_esnli],
            'faithfulness': {
                'comprehensiveness': comp_list_esnli[1],
                'macro_comprehensiveness': float(np.mean(comp_list_esnli[1])),
            },
            'plausibility': {
                'iou': iou_scores_esnli[1],
                'macro_iou': float(np.mean(iou_scores_esnli[1])),
            },
        },
        'base': {
            'explanations_as_list':[explanation.as_list(label= explanation.top_labels[0]) for explanation in explanations_base_esnli],
            'faithfulness': {
                'comprehensiveness': comp_list_esnli[2],
                'macro_comprehensiveness': float(np.mean(comp_list_esnli[2])),
            },
            'plausibility': {
                'iou': iou_scores_esnli[2],
                'macro_iou': float(np.mean(iou_scores_esnli[2])),
            },
        },
        'large': {
            'explanations_as_list':[explanation.as_list(label= explanation.top_labels[0]) for explanation in explanations_large_esnli],
            'faithfulness': {
                'comprehensiveness': comp_list_esnli[3],
                'macro_comprehensiveness': float(np.mean(comp_list_esnli[3])),
            },
            'plausibility': {
                'iou': iou_scores_esnli[3],
                'macro_iou': float(np.mean(iou_scores_esnli[3])),
            },
        },
}

In [ ]:
explainability_results['cose'] = {
        #'dataset': dataset_cose,
        'xsmall': {
            'explanations_as_list':[explanation.as_list(label= explanation.top_labels[0]) for explanation in explanations_xsmall_cose],
            'faithfulness': {
                'comprehensiveness': comp_list_cose[0],
                'macro_comprehensiveness': np.mean(comp_list_cose[0]),
            },
            'plausibility': {
                'iou': iou_scores_cose[0],
                'macro_iou': np.mean(iou_scores_cose[0]),
            },
        },
        'small': {
            'explanations_as_list':[explanation.as_list(label= explanation.top_labels[0]) for explanation in explanations_small_cose],
            'faithfulness': {
                'comprehensiveness': comp_list_cose[1],
                'macro_comprehensiveness': np.mean(comp_list_cose[1]),
            },
            'plausibility': {
                'iou': iou_scores_cose[1],
                'macro_iou': np.mean(iou_scores_cose[1]),
            },
        },
        'base': {
            'explanations_as_list':[explanation.as_list(label= explanation.top_labels[0]) for explanation in explanations_base_cose],
            'faithfulness': {
                'comprehensiveness': comp_list_cose[2],
                'macro_comprehensiveness': np.mean(comp_list_cose[2]),
            },
            'plausibility': {
                'iou': iou_scores_cose[2],
                'macro_iou': np.mean(iou_scores_cose[2]),
            },
        },
#        'large': {
#            'explanations_as_list':[explanation.as_list(label= explanation.top_labels[0]) for explanation in explanations_large_cose],
#            'explanations_as_html':[explanation.as_html(label= explanation.top_labels[0]) for explanation in explanations_large_cose],
#            'faithfulness': {
#                'comprehensiveness': comp_list_cose[3],
#                'macro_comprehensiveness': np.mean(comp_list_cose[3]),
#            },
#            'plausibility': {
#                'iou': iou_scores_cose[3],
#                'macro_iou': np.mean(iou_scores_cose[3]),
#            },
#        },
}



In [ ]:
explainability_results = {}
explainability_results['cose'] = {
        #'dataset': dataset_cose,
        'large': {
            'explanations_as_list':[explanation.as_list(label= explanation.top_labels[0]) for explanation in explanations_large_cose],
            'faithfulness': {
                'comprehensiveness': comp_list_cose[0],
                'macro_comprehensiveness': np.mean(comp_list_cose[0]),
            },
            'plausibility': {
                'iou': iou_scores_cose[0],
                'macro_iou': np.mean(iou_scores_cose[0]),
            },
        }
}


In [ ]:
# converting np.float32 types to float for serializing
def check_and_convert_types(obj):
    if isinstance(obj, dict):
        for key, value in obj.items():
            obj[key] = check_and_convert_types(value)
    elif isinstance(obj, list):
        for i in range(len(obj)):
            obj[i] = check_and_convert_types(obj[i])
    elif isinstance(obj, np.float32):  # Replace np.float32 with whatever type you want to check for
        return float(obj)  # Convert to Python native float
    return obj

explainability_results = check_and_convert_types(explainability_results)

In [ ]:
import json

# Write to file
with open('explainability_results_esnli.json', 'w') as f:
    json.dump(explainability_results, f)


In [ ]:
import json

# Write to file
with open('explainability_results_cose_large_2.json', 'w') as f:
    json.dump(explainability_results, f)


In [ ]:
from google.colab import files

files.download('explainability_results_cose_large_2.json')


In [ ]:
def print_key_tree(d, indent=0):
    for key, value in d.items():
        print('  ' * indent + str(key))
        if isinstance(value, dict):
            print_key_tree(value, indent + 1)
        elif isinstance(value, list):
            if all(isinstance(i, dict) for i in value):
                for sub_dict in value:
                    print_key_tree(sub_dict, indent + 1)

print_key_tree(explainability_results)


In [ ]:
from google.colab import files

files.download('explainability_results_esnli.json')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Logging predictions

In [ ]:
%%time

predictions_mnli, predictions_esnli, predictions_cose = [],[],[]

for model in models_zsc:
  result = model.get_results(dataset_cose['question'], candidate_labels_list=dataset_cose['candidate_labels_list'])
  pred = model.get_predictions(result, dataset_cose['candidate_labels_list'])
  predictions_cose.append(pred)



CPU times: user 20min 52s, sys: 3.2 s, total: 20min 55s
Wall time: 5min 13s


In [ ]:
for model in models_nli:
  pred_mnli = model.get_predictions(dataset_mnli['sentence_pairs'])
  pred_esnli = model.get_predictions(dataset_esnli['sentence_pairs'])
  predictions_mnli.append(pred_mnli)
  predictions_esnli.append(pred_esnli)

In [ ]:
import json
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

results = {
    'esnli': {
        'true': dataset_esnli['test_labels'],
          'xsmall': {
              'pred': predictions_esnli[0],
              'accuracy': accuracy_score(dataset_esnli['test_labels'], predictions_esnli[0]),
              'percision': precision_score(dataset_esnli['test_labels'], predictions_esnli[0], average='macro'),
              'recall': recall_score(dataset_esnli['test_labels'], predictions_esnli[0], average='macro'),
              'f1': f1_score(dataset_esnli['test_labels'], predictions_esnli[0], average='macro'),
          },
          'small': {
              'pred': predictions_esnli[1],
              'accuracy': accuracy_score(dataset_esnli['test_labels'], predictions_esnli[1]),
              'percision': precision_score(dataset_esnli['test_labels'], predictions_esnli[1], average='macro'),
              'recall': recall_score(dataset_esnli['test_labels'], predictions_esnli[1], average='macro'),
              'f1': f1_score(dataset_esnli['test_labels'], predictions_esnli[1], average='macro'),
          },
          'base': {
              'pred': predictions_esnli[2],
              'accuracy': accuracy_score(dataset_esnli['test_labels'], predictions_esnli[2]),
              'percision': precision_score(dataset_esnli['test_labels'], predictions_esnli[2], average='macro'),
              'recall': recall_score(dataset_esnli['test_labels'], predictions_esnli[2], average='macro'),
              'f1': f1_score(dataset_esnli['test_labels'], predictions_esnli[2], average='macro'),
          },
          'large': {
              'pred': predictions_esnli[3],
              'accuracy': accuracy_score(dataset_esnli['test_labels'], predictions_esnli[3]),
              'percision': precision_score(dataset_esnli['test_labels'], predictions_esnli[3], average='macro'),
              'recall': recall_score(dataset_esnli['test_labels'], predictions_esnli[3], average='macro'),
              'f1': f1_score(dataset_esnli['test_labels'], predictions_esnli[3], average='macro'),
          },
    },
        'mnli': {
          'true': dataset_mnli['test_labels'],
          'xsmall': {
              'pred': predictions_mnli[0],
              'accuracy': accuracy_score(dataset_mnli['test_labels'], predictions_mnli[0]),
              'percision': precision_score(dataset_mnli['test_labels'], predictions_mnli[0], average='macro'),
              'recall': recall_score(dataset_mnli['test_labels'], predictions_mnli[0], average='macro'),
              'f1': f1_score(dataset_mnli['test_labels'], predictions_mnli[0], average='macro'),
          },
          'small': {
              'pred': predictions_mnli[1],
              'accuracy': accuracy_score(dataset_mnli['test_labels'], predictions_mnli[1]),
              'percision': precision_score(dataset_mnli['test_labels'], predictions_mnli[1], average='macro'),
              'recall': recall_score(dataset_mnli['test_labels'], predictions_mnli[1], average='macro'),
              'f1': f1_score(dataset_mnli['test_labels'], predictions_mnli[1], average='macro'),
          },
          'base': {
              'pred': predictions_mnli[2],
              'accuracy': accuracy_score(dataset_mnli['test_labels'], predictions_mnli[2]),
              'percision': precision_score(dataset_mnli['test_labels'], predictions_mnli[2], average='macro'),
              'recall': recall_score(dataset_mnli['test_labels'], predictions_mnli[2], average='macro'),
              'f1': f1_score(dataset_mnli['test_labels'], predictions_mnli[2], average='macro'),
          },
          'large': {
              'pred': predictions_mnli[3],
              'accuracy': accuracy_score(dataset_mnli['test_labels'], predictions_mnli[3]),
              'percision': precision_score(dataset_mnli['test_labels'], predictions_mnli[3], average='macro'),
              'recall': recall_score(dataset_mnli['test_labels'], predictions_mnli[3], average='macro'),
              'f1': f1_score(dataset_mnli['test_labels'], predictions_mnli[3], average='macro'),
          },
    },
        'cose': {
          'true': dataset_cose['true_labels'],
          'xsmall': {
              'pred': predictions_cose[0],
              'accuracy': accuracy_score(dataset_cose['true_labels'], predictions_cose[0]),
          },
          'small': {
              'pred': predictions_cose[1],
              'accuracy': accuracy_score(dataset_cose['true_labels'], predictions_cose[1]),
          },
          'base': {
              'pred': predictions_cose[2],
              'accuracy': accuracy_score(dataset_cose['true_labels'], predictions_cose[2]),
          },
          'large': {
              'pred': predictions_cose[3],
              'accuracy': accuracy_score(dataset_cose['true_labels'], predictions_cose[3]),
          },
    },
}

# results are of type np.int64 which is not serializable so we convert it to int()
def convert_numpy_int(item):
    if isinstance(item, np.int64):
        return int(item)
    elif isinstance(item, list):
        return [convert_numpy_int(sub_item) for sub_item in item]
    elif isinstance(item, dict):
        return {key: convert_numpy_int(value) for key, value in item.items()}
    else:
        return item

results_converted = convert_numpy_int(results)


# Save the dictionary as a JSON file
with open('predictions_for_explanations.json', 'w') as json_file:
    json.dump(results_converted, json_file, indent=4)

from google.colab import files

files.download('predictions_for_explanations.json')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>